In [38]:
from dotenv import load_dotenv
import os
from langchain.sql_database import SQLDatabase
import google.generativeai as ai
load_dotenv()

True

In [39]:
ai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

In [41]:
#function to laod google gemini model and provide sql query for response

def get_gemin_response(question,propmpt):
    model = ai.GenerativeModel('gemini-pro')
    response = model.generate_content([propmpt,question])
    return response.text

In [16]:
import mysql.connector as connector
cnx = connector.connect(user='root',
                        password='root',
                        host='localhost',
                        database='lab2')

cursor = cnx.cursor()
result = cursor.execute("SELECT * FROM emp")
rows = cursor.fetchall()
for row in rows:
    print(row)
cnx.close()


(7369, 'SMITH', 'CLERK', 7902, datetime.date(1980, 12, 17), Decimal('800.00'), None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.date(1981, 2, 20), Decimal('1600.00'), Decimal('300.00'), 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.date(1981, 2, 22), Decimal('1250.00'), Decimal('500.00'), 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.date(1981, 4, 2), Decimal('2975.00'), None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.date(1981, 9, 28), Decimal('1250.00'), Decimal('1400.00'), 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.date(1981, 5, 1), Decimal('2850.00'), None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.date(1981, 6, 9), Decimal('2450.00'), None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.date(1987, 4, 19), Decimal('3000.00'), None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.date(1981, 11, 17), Decimal('5000.00'), None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.date(1981, 9, 8), Decimal('1500.00'), Decimal('0.00'), 30)
(7876, 'ADAMS', 'CLERK', 7788, datetim

In [42]:
## Function to retrive query from sql database 

def read_sql_query(sql):
    cnx = connector.connect(user='root',
                            password='root',
                            host='localhost',
                            database='mobile')
    cursor = cnx.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    cnx.close()
    # for row in rows:
    #     print(row)
    return rows



In [45]:
prompt ="""
You are an expert in converting English questions to SQL queries!
The SQL database has the name 'MOBILE' and the following columns: 
Brand, Model, Storage, RAM, Screen Size (inches), Camera (MP), 
Battery Capacity (mAh), Price, Min_price.

Here are some example translations:

Example 1:
Question: "How many mobile entries are in the database?"
SQL: SELECT COUNT(*) FROM MOBILE;

Example 2:
Question: "Show me all the mobiles from the brand Apple."
SQL: SELECT * FROM MOBILE WHERE BRAND = 'Apple';

Example 3:
Question: "List all mobiles under $500."
SQL: SELECT * FROM MOBILE WHERE PRICE < 500;

Instructions:
- Translate each English question into an SQL query based on the provided table structure.
- Do not include `'''` at the beginning or end of the SQL code.
- Do not use the word 'SQL' in the response.
- Ensure that the generated SQL queries are syntactically correct.
""" 

In [54]:
ques = 'top 5 most expensvie samsumg phones'
response = get_gemin_response(propmpt=prompt,question=ques)
print(response)


SELECT * FROM MOBILE WHERE BRAND = 'Samsung' ORDER BY PRICE DESC LIMIT 5;


In [55]:
result = read_sql_query(response)
result

[('Samsung',
  'Galaxy S20 Ultra 5G',
  '512',
  '16',
  6.9,
  '108+48+12+3D',
  5000,
  1399),
 ('Samsung',
  'Galaxy Note 20 Ultra',
  '256',
  '12',
  6.9,
  '108+12+12',
  4500,
  1299),
 ('Samsung',
  'Galaxy S21 Ultra',
  '256 GB',
  '12 GB',
  6.8,
  '108 + 10 + 10 + 12',
  5000,
  1199),
 ('Samsung',
  'Galaxy S21 Ultra 5G',
  '256',
  '12',
  6.8,
  '108+10+10+12',
  5000,
  1199),
 ('Samsung',
  'Galaxy S20 Ultra 5G',
  '128',
  '12',
  6.9,
  '108+48+12+0.3',
  5000,
  1199)]

In [58]:
def sql_result_to_text(column_names, sql_result):
    """
    Convert the SQL result to plain text dynamically based on the column names and result data.
    
    Args:
        column_names (list): List of column names returned by the SQL query.
        sql_result (list of tuples): The actual result data (each tuple represents a row).
    
    Returns:
        str: Plain text representation of the SQL result.
    """
    
    formatted_rows = []
    
    
    for row in sql_result:
        
        row_text = []
        
        for col_name, value in zip(column_names, row):
            row_text.append(f"{col_name}: {value}")
        
        formatted_rows.append(", ".join(row_text))
    
    
    return "\n".join(formatted_rows)

# Example usage
column_names = ['Brand','Model','Storage','RAM','Screen Size (inches)','Camera (MP)','Battery Capacity (mAh)','Price']  # This would be dynamic in your case
sql_result = result

# Convert the SQL result to plain text
result_text = sql_result_to_text(column_names, sql_result)
print(result_text)


Brand: Samsung, Model: Galaxy S20 Ultra 5G, Storage: 512, RAM: 16, Screen Size (inches): 6.9, Camera (MP): 108+48+12+3D, Battery Capacity (mAh): 5000, Price: 1399
Brand: Samsung, Model: Galaxy Note 20 Ultra, Storage: 256, RAM: 12, Screen Size (inches): 6.9, Camera (MP): 108+12+12, Battery Capacity (mAh): 4500, Price: 1299
Brand: Samsung, Model: Galaxy S21 Ultra, Storage: 256 GB, RAM: 12 GB, Screen Size (inches): 6.8, Camera (MP): 108 + 10 + 10 + 12, Battery Capacity (mAh): 5000, Price: 1199
Brand: Samsung, Model: Galaxy S21 Ultra 5G, Storage: 256, RAM: 12, Screen Size (inches): 6.8, Camera (MP): 108+10+10+12, Battery Capacity (mAh): 5000, Price: 1199
Brand: Samsung, Model: Galaxy S20 Ultra 5G, Storage: 128, RAM: 12, Screen Size (inches): 6.9, Camera (MP): 108+48+12+0.3, Battery Capacity (mAh): 5000, Price: 1199
